In [1]:
cd E:/GitHub/computervision

E:\GitHub\computervision


In [2]:
import torch
import os
# import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader

from modules.models_util import EfficientNet, ViT, VGG19, Erase
from modules.training_util import PytorchDataset

torch.manual_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights_path = [
    'notebooks/efficientnet/models/2024-02-01 10-31-27/best.pth',       # Fully trained 4k, 53.10%, 52.82% without transforms
    'notebooks/efficientnet/models/2024-02-01 09-55-37/best.pth',       # Fully trained 2k, 52.74%, 58.21%
    'notebooks/efficientnet/models/2024-02-02 16-34-31/best.pth',       # Fully trained even, 50.91%, 45.99%
    'notebooks/efficientnet/models/2024-31-01 18-53-33/best.pth',       # 4k, 55.84%, 53.83%
    'notebooks/efficientnet/models/2024-31-01 19-19-52/best.pth',       # 2k, 49.82%, 52.55%
    'notebooks/vit/models/01-31-2024 15-18-02/epoch_12.pth',            # ViT, 61.86%, 55.71% 8-1-1, 55.5%, 63% cor dataset
    'notebooks/vit/models/2024-02-03 21-45-07/best.pth',                # 60.4%, 52% new dataset
    'notebooks/vit/models/2024-02-03 21-53-43/best.pth',                # 48.54%
    'notebooks/vgg19/models/2024-02-04 16-46-29/epoch_15.pth',          # 55%
    'notebooks/efficientnet/models/2024-02-05 16-41-35/epoch_15.pth',   # 45.44%, 49% in same dataset
    'notebooks/efficientnet/models/2024-02-06 17-36-09/best.pth',
    'notebooks/vit/models/2024-02-06 19-21-50/best.pth'                 # 
    
]
# model = EfficientNet().load(weights_path[-1].replace('\\','/'))
model = ViT().load(weights_path[-1].replace('\\','/'))
# model = VGG19.load(weights_path[8].replace('\\','/'))
model.to(device)
model.eval()

All parameters for model VisionTransformer requires grad.
Weights for model VisionTransformer loaded from notebooks/vit/models/2024-02-06 19-21-50/best.pth


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [4]:
# dataset_dir = 'E:/Datasets/cor-splits/temp-test'
dataset_dir = 'E:/Datasets/cor-splits/sgkf-8-1-1-size-1010'
# dataset_dir = 'E:/Datasets/cor'

data_transforms = ViT().data_transforms
dataset = PytorchDataset(dataset_dir, data_transforms, training_phases=['test'])
dataloader = dataset.dataloaders['test']

In [9]:
dataset.codes['test']

{'CODE1475', 'CODE326', 'CODE38'}

In [5]:
dataloader.dataset.class_to_idx

{'0': 0, '1': 1}

In [33]:
from sklearn.metrics import confusion_matrix

correct_predictions = 0
total_samples = 0

n_cls = dataset.get_n_classes()

acc = [0 for _ in range(n_cls)]

codelist = {x:[0,0] for x in dataset.codes['test']}

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad(): # no backtracking
    for i, (inputs, labels, paths) in enumerate(dataloader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        
        _, predicted = torch.max(outputs, 1)
        
        for j, path in enumerate(paths):
            code = os.path.basename(path).split(' ')[0]
            codelist[code][0] += (predicted[j] == labels[j]).sum().item()
            codelist[code][1] += 1
        
        # Append batch prediction results
        predlist=torch.cat([predlist,predicted.view(-1).cpu()])
        lbllist=torch.cat([lbllist,labels.view(-1).cpu()])
        
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print('Confusion Matrix')
print(conf_mat)

# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print('Accuracy per class')
print(class_accuracy)

accuracy = correct_predictions/total_samples
print(f'\nAccuracy: {accuracy * 100:.2f}%')

Confusion Matrix
[[31 17]
 [14 34]]
Accuracy per class
[64.58333333 70.83333333]

Accuracy: 67.71%


In [34]:
codelist

{'CODE1475': [18, 28], 'CODE326': [27, 34], 'CODE38': [20, 34]}

In [35]:
code_acc = {code:codelist[code][0]/codelist[code][1] for code in codelist.keys()}
code_acc

{'CODE1475': 0.6428571428571429,
 'CODE326': 0.7941176470588235,
 'CODE38': 0.5882352941176471}

In [37]:
average = sum(code_acc.values())/len(code_acc)
average

0.6750700280112044